In [1]:
import base64
import os
import json # Import the json library to read the file
from google import genai
from google.genai import types


In [ ]:
def generate(query):
    # --- 1. Load the JSON File ---
    # Define the path to your knowledge base file
    hpo_file_path = "hpo_hits.json" 
    
    try:
        with open(hpo_file_path, 'r', encoding='utf-8') as f:
            hpo_data = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{hpo_file_path}' was not found. Please ensure it is in the same directory.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return
    # -------------------------------

    client = genai.Client(
        api_key="",
    )

    model = "gemini-2.5-flash"
    
    # --- 2. Construct the Contents List with the File Data ---
    contents = [
        types.Content(
            role="user",
            parts=[
                # Add the HPO data file as a text part
                types.Part.from_text(text=hpo_data), 
                # Add the user's main input/prompt
                types.Part.from_text(text=query),
            ],
        ),
    ]
    # --------------------------------------------------------

    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        system_instruction=[
            types.Part.from_text(text="""You are an expert biomedical text analysis agent. Your primary function is to identify and extract clinical phenotypes from the user's text and map them to the most relevant Human Phenotype Ontology (HPO) terms.

The first part of the user input is a JSON object containing the HPO knowledge base. Use this knowledge base for your task. Pay close attention to the labels, definitions, and synonyms to find the best semantic match.
"""),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")



In [3]:
query="at the age of 22, 5 years this German woman experienced muscle cramps, progressive paresis and muscle wasting beginning in one leg and spreading to the other leg without sensory, autonomic or cognitive symptoms. A myopathy was initially suspected but EMG revealed acute and chronic signs of denervation mostly in the legs but also slightly in the upper limbs. Transcortical magnetic evoked potential (MEP) examination was pathological with delayed central conductance time to both lower limbs but not to the upper limbs. Peripheral nerve conduction studies were normal. The medical history was unremarkable except for a voluntarily weight loss of ≈10 kg in the years before appearance of muscle symptoms. Both the parents have diabetes mellitus type 2, but there is no family history of a neuromuscular disease (NMD) or FTD-like condition. The patient eventually received a diagnosis of sALS. The disease has developed slowly, symptoms and signs of upper motor neuron (UMN) and lower motor neuron (LMN) damage only appearing in the upper limbs 3 years after onset in the lower limbs. Nine years after onset of paresis, the patient is alive and still show no bulbar symptoms."
generate(query)

```json
[
    {
        "phenotype": "progressive paresis",
        "hpo_id": "HP:0002385",
        "hpo_label": "Paraparesis"
    },
    {
        "phenotype": "muscle wasting",
        "hpo_id": "HP:0003202",
        "hpo_label": "Skeletal muscle atrophy"
    },
    {
        "phenotype": "EMG revealed acute and chronic signs of denervation",
        "hpo_id": "HP:0003444",
        "hpo_label": "EMG: chronic denervation signs"
    },
    {
        "phenotype": "delayed central conductance time to both lower limbs",
        "hpo_id": "HP:0012898",
        "hpo_label": "Abnormal lower-limb motor evoked potentials"
    },
    {
        "phenotype": "upper motor neuron (UMN) damage",
        "hpo_id": "HP:0002493",
        "hpo_label": "Upper motor neuron dysfunction"
    },
    {
        "phenotype": "lower motor neuron (LMN) damage ... appearing in the upper limbs",
        "hpo_id": "HP:0003484",
        "hpo_label": "Upper limb muscle weakness"
    },
    {
        "phenotype": "lowe